In [ ]:
import pybullet as p

from full_demo import create_env
import time
from pybullet_robots.panda.panda_sim_grasp import rp, pandaEndEffectorIndex

In [ ]:
simId, robotId, ll, ul, jr, bounds, tableheight = create_env(with_block=False, bad_start=True)


In [ ]:
# Let's say we have a goal in mind!

# Specify the goal.
pos = [0, 0, tableheight + 0.2]
orient = [1, 0, 0, 0]

# Perform the inverse kinematics.
# Note: this function tries to find the best IK solution given your CURRENT position in the env.
# Since there are many links in the panda chain, we are computing the desired pose of the final link, a 
# virtual position between the fingers called pandaEndEffectorIndex here.
goal_ja = p.calculateInverseKinematics(robotId, pandaEndEffectorIndex, pos, orient, ll, ul, jr, rp, maxNumIterations=1000)

In [ ]:
# FULL STEAM AHEAD TO THE GOAL!
for i in range(10000):
    # Set the control values
    p.setJointMotorControlArray(robotId, jointIndices=[0,1,2,3,4,5,6], controlMode=p.POSITION_CONTROL, targetPositions=goal_ja[:7])
    p.stepSimulation()
    time.sleep(1./240.)

In [ ]:
# I'm actually being a bit dishonest here........
# The initial starting position is actually invalid, and so the simulator has a hard time actually driving things there.
# You ***might*** get lucky with your starting conditions, and sending things directly to the goal joint angles might work.
# BUT IN THE GENERAL CASE, you're not guaranteed that your motion will be nice. Things that could go wrong:
# * self-collisions as the robot is driving its joints
# * end effector might move in unexpected ways due to the power of the motors
# * collisions with other objects make things infeasible

# WE WANT SOME MORE GUARANTEES!